In [1]:
import pandas as pd
from pandasgui import show
import numpy as np
import matplotlib.pyplot as plt


# Preprocessing part of the problem statement

In [2]:
# get the data at global level
confirmed_cases_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recover_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [3]:
#show(confirmed_cases_raw)

![title](plots/newplot_confirmed_cases.png)

In [4]:
#show(death_raw)

![title](plots/newplot_death.png)

In [5]:
#show(recover_raw)

![title](plots/newplot_recover.png)

In [6]:
confirmed_cases_raw.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,49621,49681,49817,50013,50190,50433,50655,50810,50886,51039
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,52004,52542,53003,53425,53814,54317,54827,55380,55755,56254
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,94371,94781,95203,95659,96069,96549,97007,97441,97857,98249
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,7519,7560,7577,7602,7633,7669,7699,7756,7806,7821
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,16562,16626,16644,16686,16802,16931,17029,17099,17149,17240


In [7]:
confirmed_cases_raw.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '12/18/20', '12/19/20', '12/20/20', '12/21/20', '12/22/20', '12/23/20',
       '12/24/20', '12/25/20', '12/26/20', '12/27/20'],
      dtype='object', length=345)

## Cleaning the raw data

In [8]:
# using melt and reiterating the dataframe
def clean_data(raw):
    df_clean = raw.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_name='Cases',var_name='Date')
    df_clean = df_clean.set_index(['Country/Region','Province/State','Date'])
    return df_clean

confirmed_cases = clean_data(confirmed_cases_raw)
death = clean_data(death_raw)
recover = clean_data(recover_raw)

In [9]:
confirmed_cases.head(5)

,,,Lat,Long,Cases
Country/Region,Province/State,Date,,,
Afghanistan,NaN,1/22/20,33.93911,67.709953,0
Albania,NaN,1/22/20,41.15330,20.168300,0
Algeria,NaN,1/22/20,28.03390,1.659600,0
Andorra,NaN,1/22/20,42.50630,1.521800,0
Angola,NaN,1/22/20,-11.20270,17.873900,0


## Will be only looking at country level and not province or state level

In [10]:
## Get Countrywise Data
def countrydata(df_clean,oldname,newname):
    df_country=df_clean.groupby(['Country/Region','Date'])['Cases'].sum().reset_index()
    df_country=df_country.set_index(['Country/Region','Date'])
    df_country.index=df_country.index.set_levels([df_country.index.levels[0], pd.to_datetime(df_country.index.levels[1])])
    df_country=df_country.sort_values(['Country/Region','Date'],ascending=True)
    df_country=df_country.rename(columns={oldname:newname})
    return df_country

new_confirmed_cases_country = countrydata(confirmed_cases, 'Cases','Total Confirmed Cases')
new_death_country = countrydata(death, 'Cases','Total Deaths')
new_recover_country = countrydata(recover, 'Cases','Total Recoveries')
#new_confirmed_cases_country.head(5)

In [11]:
## Get DailyData from Cumulative sum
def dailydata(dfcountry,oldname,newname):
    dfcountrydaily=dfcountry.groupby(level=0).diff().fillna(0)
    dfcountrydaily=dfcountrydaily.rename(columns={oldname:newname})
    return dfcountrydaily


new_CasesCountry=dailydata(new_confirmed_cases_country,'Total Confirmed Cases','Daily New Cases')
new_DeathsCountry=dailydata(new_death_country,'Total Deaths','Daily New Deaths')
new_RecoveriesCountry=dailydata(new_recover_country,'Total Recoveries','Daily New Recoveries')
#new_CasesCountry.head(5)

In [12]:
Country_Consolidated=pd.merge(new_confirmed_cases_country,new_CasesCountry,how='left',left_index=True,right_index=True)
Country_Consolidated=pd.merge(Country_Consolidated,new_DeathsCountry,how='left',left_index=True,right_index=True)
Country_Consolidated=pd.merge(Country_Consolidated,new_death_country,how='left',left_index=True,right_index=True)
Country_Consolidated=pd.merge(Country_Consolidated,new_recover_country,how='left',left_index=True,right_index=True)
Country_Consolidated=pd.merge(Country_Consolidated,new_RecoveriesCountry,how='left',left_index=True,right_index=True)
Country_Consolidated['Active Cases']=Country_Consolidated['Total Confirmed Cases']-Country_Consolidated['Total Deaths']-Country_Consolidated['Total Recoveries']
Country_Consolidated['Share of Recoveries - Closed Cases']=np.round(Country_Consolidated['Total Recoveries']/(Country_Consolidated['Total Recoveries']+Country_Consolidated['Total Deaths']),2)
Country_Consolidated['Death to Cases Ratio']=np.round(Country_Consolidated['Total Deaths']/Country_Consolidated['Total Confirmed Cases'],3)

In [13]:
GlobalTotals=Country_Consolidated.reset_index().groupby('Date').sum()
GlobalTotals['Share of Recoveries - Closed Cases']=np.round(GlobalTotals['Total Recoveries']/(GlobalTotals['Total Recoveries']+GlobalTotals['Total Deaths']),2)
GlobalTotals['Death to Cases Ratio']=np.round(GlobalTotals['Total Deaths']/GlobalTotals['Total Confirmed Cases'],3)
GlobalTotals.tail(2)

,Total Confirmed Cases,Daily New Cases,Daily New Deaths,Total Deaths,Total Recoveries,Daily New Recoveries,Active Cases,Share of Recoveries - Closed Cases,Death to Cases Ratio
Date,,,,,,,,,
2020-12-26,80351598,511392.0,7077.0,1757657,45410291,407732.0,33183650,0.96,0.022
2020-12-27,80783674,432076.0,7206.0,1764863,45698074,287783.0,33320737,0.96,0.022


In [14]:
GlobalTotals.iloc[-1]

Total Confirmed Cases                 8.078367e+07
Daily New Cases                       4.320760e+05
Daily New Deaths                      7.206000e+03
Total Deaths                          1.764863e+06
Total Recoveries                      4.569807e+07
Daily New Recoveries                  2.877830e+05
Active Cases                          3.332074e+07
Share of Recoveries - Closed Cases    9.600000e-01
Death to Cases Ratio                  2.200000e-02
Name: 2020-12-27 00:00:00, dtype: float64